<a href="https://colab.research.google.com/github/MatiasSiles/Deteccion-de-Fraudes-en-Blockchain/blob/main/EntregaN1_SilesMatias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Deteccion de Fraudes en Blockchain**

In [3]:
import requests as r
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as w
w.filterwarnings("ignore")

##**Motivacion/Audiencia**

Este proyecto busca abordar uno de los problemas más críticos en el ecosistema blockchain: las estafas en las transacciones de criptomonedas. A través del análisis de datos en la cadena de bloques, se pretende identificar patrones sospechosos que podrían comprometer la seguridad y la estabilidad a largo plazo del mercado cripto. Este enfoque no solo beneficia a los inversionistas de criptomonedas, sino también a los profesionales de la seguridad que trabajan para proteger estos activos digitales.

##**Preguntas/Hipotesis**

1. Cuales son los patrones mas comunes en transacciones fraudulentas en blockchain?

2. Que características de las transacciones indican un posible fraude?

3. Que porcentaje de transacciones fraudulentas se detecta en tiempo real?

4. Que monedas digitales presentaron fraudes?

5. Como afectan los fraudes a la volatilidad de las criptomonedas?

**Hipotesis**: Los casos de fraude en criptomonedas aumentan durante eventos significativos, como el lanzamiento de nuevas criptomonedas, y en periodos de alta volatilidad del mercado

#**EDA**

##Lectura desde la API CoinGecko

In [4]:
url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd"
respuesta = r.get(url)

if respuesta.status_code == 200:
  datos = respuesta.json()
  df_criptos = pd.DataFrame(datos)
else:
  print(f"Error: {respuesta.status_code}")

##Data Wrangling

Descripciones Breves de las Columnas

1. symbol = Abreviatura o alias de la criptomoneda

2. current_price = Precio actual de la moneda en USD

3. market_cap_billion = Capitalización de mercado en miles de millones (USD)

4. market_cap_rank = Posición en el ranking por capitalización

5. fully_diluted_valuation_billion = Valor proyectado si todas las monedas estuvieran en circulación

6. total_volume_billion = Volumen total de operaciones en las últimas 24h (en miles de millones USD)

7. high_24h = Precio más alto registrado en las últimas 24hs

8. low_24h = Precio más bajo registrado en las últimas 24hs

9. price_change_24h = Cambio absoluto del precio en las últimas 24hs

10. price_change_percentage_24h = Cambio porcentual del precio en 24hs

11. market_cap_change_24h_billion = Variación de capitalización de mercado en 24hs (miles de millones)

12. market_cap_change_percentage_24h = Porcentaje de variación del market cap en 24hs

13. circulating_supply_billion = Oferta actual de monedas en circulación (miles de millones)

14. total_supply_billion = Suministro total estimado (miles de millones)

15. ath = Precio maximo historico ("All Time High")

16. ath_change_percentage = Porcentaje que ha bajado desde su maximo historico.

17. ath_date = Fecha del precio maximo historico

18. atl = Precio minimo historico ("All Time Low").

19. atl_change_percentage = Porcentaje que ha subido desde su minimo historico

20. atl_date = Fecha del precio minimo historico

21. last_updated = Ultima fecha y hora de actualización de los datos.

###1. Descubrimiento

In [5]:
df_criptos.head(5)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,106335.00,2113490562229,1,2113490562229,1.534411e+10,106365.00,...,1.987615e+07,2.100000e+07,111814.00,-4.87947,2025-05-22T18:41:28.492Z,67.810000,1.567497e+05,2013-07-06T00:00:00.000Z,None,2025-06-08T19:24:24.832Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2536.03,306130345424,2,306130345424,9.350259e+09,2540.39,...,1.207231e+08,NaN,4878.26,-48.01543,2021-11-10T14:24:19.604Z,0.432979,5.855966e+05,2015-10-20T00:00:00.000Z,"{'times': 30.88451297484912, 'currency': 'btc'...",2025-06-08T19:24:25.064Z
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.00,154820589807,3,154820589807,2.836483e+10,1.00,...,1.547608e+11,NaN,1.32,-24.39024,2018-07-24T00:00:00.000Z,0.572521,7.473405e+01,2015-03-02T00:00:00.000Z,None,2025-06-08T19:24:21.648Z
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.29,134596508305,4,228789468548,2.067060e+09,2.29,...,9.998606e+10,1.000000e+11,3.40,-32.69177,2018-01-07T00:00:00.000Z,0.002686,8.505490e+04,2014-05-22T00:00:00.000Z,None,2025-06-08T19:24:24.829Z
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,654.06,95414842373,5,95414842373,3.789735e+08,653.99,...,1.458876e+08,2.000000e+08,788.84,-17.09567,2024-12-04T10:35:25.220Z,0.039818,1.642349e+06,2017-10-19T00:00:00.000Z,None,2025-06-08T19:24:26.049Z


In [6]:
# Informacion Irrelevante
df_criptos.drop(columns=["id","name","image"], inplace=True)

In [7]:
# Valores unicos
df_criptos.select_dtypes(exclude=["object"]).nunique()

,0
current_price,97
market_cap,100
market_cap_rank,100
fully_diluted_valuation,100
total_volume,100
high_24h,91
low_24h,97
price_change_24h,100
price_change_percentage_24h,100
market_cap_change_24h,100


In [8]:
df_criptos.describe().round(2)

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage
count,100.00,1.000000e+02,100.00,1.000000e+02,1.000000e+02,100.00,100.00,100.00,100.00,1.000000e+02,100.00,1.000000e+02,1.000000e+02,4.600000e+01,100.00,100.00,100.00,100.00
mean,5561.26,3.358455e+10,50.50,3.622551e+10,7.659971e+08,5564.86,5492.05,28.15,0.85,2.496772e+08,0.90,1.088361e+13,1.100275e+13,1.109015e+13,6114.40,-50.31,1684.84,258933.99
std,23243.25,2.135242e+11,29.01,2.140358e+11,3.356551e+09,23255.97,22957.37,117.58,2.18,1.288062e+09,2.20,7.235878e+13,7.249348e+13,6.310638e+13,24541.89,32.00,9076.56,2251595.53
min,0.00,9.725739e+08,1.00,9.725739e+08,0.000000e+00,0.00,0.00,-1.75,-2.93,-1.815518e+08,-2.57,9.156860e+03,9.156860e+03,1.288150e+05,0.00,-99.20,0.00,0.00
25%,0.67,1.450024e+09,25.75,1.899492e+09,2.032924e+07,0.67,0.65,-0.00,-0.02,-3.464611e+05,-0.02,1.240942e+08,1.506501e+08,3.250000e+08,1.13,-78.60,0.09,55.26
50%,2.16,2.590235e+09,50.50,3.930344e+09,6.854740e+07,2.18,2.08,0.00,0.43,1.066070e+07,0.48,1.001907e+09,1.138909e+09,1.409000e+09,5.21,-57.38,0.58,480.46
75%,32.55,6.699529e+09,75.25,8.673868e+09,1.532563e+08,32.68,32.08,0.23,1.12,4.887238e+07,1.20,4.979369e+09,8.964811e+09,1.000000e+10,184.53,-17.77,1.35,7160.08
max,106367.00,2.113491e+12,100.00,2.113491e+12,2.836483e+10,106555.00,105147.00,646.16,16.09,1.102984e+10,16.20,5.892494e+14,5.895039e+14,4.206900e+14,114576.00,0.00,57439.00,22481957.48


In [9]:
print(df_criptos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   symbol                            100 non-null    object 
 1   current_price                     100 non-null    float64
 2   market_cap                        100 non-null    int64  
 3   market_cap_rank                   100 non-null    int64  
 4   fully_diluted_valuation           100 non-null    int64  
 5   total_volume                      100 non-null    float64
 6   high_24h                          100 non-null    float64
 7   low_24h                           100 non-null    float64
 8   price_change_24h                  100 non-null    float64
 9   price_change_percentage_24h       100 non-null    float64
 10  market_cap_change_24h             100 non-null    float64
 11  market_cap_change_percentage_24h  100 non-null    float64
 12  circulati

In [51]:
df_criptos.isna().sum().loc[df_criptos.isna().sum().values > 0].rename("nan")

,nan
max_supply,54
roi,88


In [63]:
df_criptos.select_dtypes(exclude="object").duplicated().sum()

np.int64(0)

Nos interesa la columna max_supply ya que tiene muchos valores duplicados, pero
visualizando la matriz de valores NaN, nos damos cuenta que no debemos eliminar sus valores duplicados,
sino, debemos eliminar la columna entera ya que tiene muchos valores faltantes, no tiene sentido imputar
la media u otro valor si tiene la columna mayor del 70% NaN.

###2. Estructuracion

In [65]:
# Billones a Miles de Millones
for columna in df_criptos.select_dtypes(exclude="object"):
  if df_criptos[columna].max() >= 1e9:
    df_criptos[columna] = df_criptos[columna] / 1e9
    df_criptos.rename(columns={columna:f"{columna}_billion"}, inplace=True)

df_criptos.head(5)

,symbol,current_price,market_cap_billion,market_cap_rank,fully_diluted_valuation_billion,total_volume_billion,high_24h,low_24h,price_change_24h,price_change_percentage_24h,...,total_supply_billion,max_supply_billion,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,btc,106335.00,2113.490562,1,2113.490562,15.344114,106365.00,105112.00,546.640000,0.51673,...,0.019876,0.021,111814.00,-4.87947,2025-05-22T18:41:28.492Z,67.810000,1.567497e+05,2013-07-06T00:00:00.000Z,None,2025-06-08T19:24:24.832Z
1,eth,2536.03,306.130345,2,306.130345,9.350259,2540.39,2496.16,15.550000,0.61693,...,0.120723,NaN,4878.26,-48.01543,2021-11-10T14:24:19.604Z,0.432979,5.855966e+05,2015-10-20T00:00:00.000Z,"{'times': 30.88451297484912, 'currency': 'btc'...",2025-06-08T19:24:25.064Z
2,usdt,1.00,154.820590,3,154.820590,28.364831,1.00,1.00,0.000055,0.00551,...,154.760836,NaN,1.32,-24.39024,2018-07-24T00:00:00.000Z,0.572521,7.473405e+01,2015-03-02T00:00:00.000Z,None,2025-06-08T19:24:21.648Z
3,xrp,2.29,134.596508,4,228.789469,2.067060,2.29,2.17,0.108759,4.99108,...,99.986060,100.000,3.40,-32.69177,2018-01-07T00:00:00.000Z,0.002686,8.505490e+04,2014-05-22T00:00:00.000Z,None,2025-06-08T19:24:24.829Z
4,bnb,654.06,95.414842,5,95.414842,0.378974,653.99,647.16,1.780000,0.27310,...,0.145888,0.200,788.84,-17.09567,2024-12-04T10:35:25.220Z,0.039818,1.642349e+06,2017-10-19T00:00:00.000Z,None,2025-06-08T19:24:26.049Z


En el dataset se observo valores muy grandes que posiblemente perjudiquen al analisis,
            por tal motivo, se transforman aquellos numeros mayores o iguales a un billon a la cifra
            de miles de millones porque es mas legible y es una escala estandar en finanzas.

###3. Limpieza

In [ ]:
# Eliminar max_supply_billion y roi
df_criptos.drop(columns=["max_supply_billion","roi"], inplace=True)



---



###4. Enriquecimiento

In [ ]:
np.random.seed(1)

df_criptos['Fraudulenta o no?'] = np.random.choice([0, 1], size=len(df_criptos), p=[0.95, 0.05])  # Aplicamos un 5% de fraude
df_criptos['Id_transaccion'] = np.random.randint(1000, 10000, size=len(df_criptos))  # Ids aleatorios
df_criptos['Fecha'] = pd.to_datetime('now') - pd.to_timedelta(np.random.randint(0, 365, size=len(df_criptos)), unit='D') # Fechas aleatorias en el último año

df_criptos['Monto'] = np.where(
    df_criptos["Fraudulenta o no?"] == 1, np.random.uniform(4.5, 5.0, size=len(df_criptos)), np.random.uniform(0.0001, 2.0, size=len(df_criptos))
) # Montos Coherentes aleatorios

df_criptos['Tipo de transaccion'] = np.random.choice(['Compra', 'Venta', 'Transferencia'], size=len(df_criptos))  # Tipos de transacción aleatorios

df_criptos.head(5)

##Visualizaciones

In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y='Monto', data=df_criptos)
plt.title('Outliers en Transacciones')
plt.ylabel('Valores de Monto')
plt.show()

In [ ]:
# Caracteristicas de las Transacciones que indican posible Fraude

plt.figure(figsize=(8,4))
plt.scatter(df_criptos["Fecha"], df_criptos["Monto"])
plt.xlabel("Fechas")
plt.ylabel("Monto de Transacciones")
plt.title("Patrones Fraudulentos en Transacciones")

plt.show()

Se observa outliers en los montos de las transacciones, las caracteristicas de patrones fraudulentos en transacciones son diversas, como valores muy elevados o muy bajos en corto tiempo. En este caso, a principios del 2025 se detectan estos patrones. Tambien ayudaria mucho investigar el horario de las transacciones fraudulentas realizadas y el tipo de transaccion (En general son ventas).

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(df_criptos["Fraudulenta o no?"].value_counts(),
        labels=["No Fraudulenta", "Fraudulenta"],
        autopct="%1.1f%%")
plt.title("Porcentajes de Transacciones Fraudulentas")

plt.show()

In [ ]:
df_criptos[df_criptos["Fraudulenta o no?"] == 1][[
    "symbol","current_price","price_change_24h","total_volume_billion","Monto","Tipo de transaccion","Fecha"]]

Estos datos de las transacciones fraudulentas permite conocer mas las razones por la cual hubo fraudes. En este caso podemos ver la volatilidad(price_change_24h), el volumen, su tipo de transaccion, etc. Datos claves para la toma de decisiones

In [ ]:
plt.figure(figsize=(4, 4))
sns.scatterplot(x='current_price', y='market_cap_billion', data=df_criptos)
plt.title('Precio vs Capitalización de Mercado(miles de millones de USD)')
plt.xlabel('Precio')
plt.ylabel('Capitalización de Mercado')
plt.show()

*  La mayoria de criptomonedas tienen bajo precio y capitalizacion, pero tambien existen criptos con alto precios y baja capitalizacion que indican tokens especulativos y el ultimo caso donde hay altos precios y capitalizacion que indican criptos dominantes en el mercado.

In [ ]:
plt.figure(figsize=(4, 4))
sns.scatterplot(x='price_change_24h', y='total_volume_billion', data=df_criptos)
plt.title('Cambio de Precio (24hs) vs Volumen de Operaciones')
plt.xlabel('Cambio de Precio en 24hs')
plt.ylabel('Volumen Total (miles de millones de USD)')
plt.show()

*   Alta variación + alto volumen = interés real
*   Alta variación + bajo volumen = manipulación o poca liquidez
*   Bajo cambio + alto volumen = acumulación o estabilidad
*   Ambos bajos = desinterés o estancamiento

Se observa valores bajos en ambas variables, por lo que podemos decir que predomina el desinteres de ciertas criptomonedas

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(df_criptos.select_dtypes(exclude='object').corr(), annot=False, cmap='coolwarm')
plt.title('Similaridad entre variables')
plt.show()

*   Se observan bastantes similitudes entre variables, pero las mas importantes y que destacan son las que ayudan a encontrar posibles fraudes entre transacciones. Entonces, podemos decir que la mayoria de variables tiene una similitud alta con las demas, por lo que de alguna forma influye en el porque se estafo en esas criptomonedas

In [ ]:
plot = sns.pairplot(df_criptos[[
    'current_price', 'market_cap_billion', 'total_volume_billion', 'price_change_percentage_24h'
]], diag_kind='hist', diag_kws={'bins': 3})

plot.fig.suptitle("Posibles Anomalías", y=1.02)  # No sobreponer el titulo en los graficos
plt.show()

#Conclusiones Finales

##Insights

*   Las criptomonedas fraudulentas son wsteth, wbt, okb, ip. Y constituyen un 4% de transacciones fraudulentas
*   Porque razones se estafo con esas Criptomonedas?: Generalmente cuando un año nuevo arranca, hay mas novatos metiendose en el mundo del blockchain, tambien por eventos significativos como "La moneda del presidente Milei que resulto una estafa"
*   2 transacciones fraudulentas son ventas, otra es transaccion y la ultima es compra. Generalmente el tipo de transaccion en estos casos, son ventas, ya que se busca estafar con precios altos
*  Los fraudes en este caso, tienen precios muy elevados y anormales, tambien podrian haber sido precios muy bajos.
*  Existen pocos casos de criptomonedas con tokens especulativos(alta volatilidad y riesgo) que su valor depende solamente de la demanda en tiempo real


##Mi sugerencia:

1. Monitoreo continuo de transacciones con características sospechosas

2. Implementar modelos predictivos

3. Visualización regular de datos para detectar anomalias tempranas

4. Validar y actualizar los datos constantemente

##Con respecto a la hipotesis:

Los eventos significativos a principios del 2025 como "$Libra de Milei" influyeron en los fraudes de las criptomonedas, porque lo que, la hipotesis cumplio ciertamente con su proposito. Tambien influyen otras y muchas variables pero una de las principales es esa.